In [ ]:
!pip install -U keras-tuner # for performing hyperparameter tuning
!pip install --upgrade tensorflow jax jaxlib # To ensure that we have the latest versions of TensorFlow and JAX installed.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 62.6 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.5.1
    Uninstalling jaxlib-0.5.1:
      Successfully uninstalled jaxlib-0.5.1
  Attempting uninstall: jax
    Found 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Dense
import keras_tuner as kf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

/usr/local/lib/python3.11/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.5.1 is installed, but it is not compatible with the installed jaxlib version 0.5.3, so it will not be used.
  warnings.warn(


In [ ]:
df = pd.read_csv('/content/Churn_Modelling.csv')

In [ ]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
df.drop(columns = ['RowNumber'	,'CustomerId',	'Surname'], inplace=True)

In [ ]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
df = pd.get_dummies(data = df,columns=['Geography','Gender'],dtype=int)
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,1,0,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,1,0,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,1,0,0,1,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1,0,0,1


In [ ]:
x = df.drop(columns = ['Exited'])
y = df['Exited']

In [ ]:
sc = StandardScaler()

x = sc.fit_transform(x)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.20, random_state=1)

In [ ]:
# Before hyperparameter tuning: initially use random parameters only for comparision
ann = Sequential()# random weights are initialized so every time you will get different accuracy and loss

ann.add(Dense(units = 10, activation = 'relu'))# hidden layer
ann.add(Dense(units = 1, activation = 'sigmoid'))# output layer

ann.compile(optimizer = 'adadelta', loss = 'binary_crossentropy', metrics =['accuracy'])

ann.fit(xtrain,ytrain,epochs = 100)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3354 - loss: 1.0244
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3466 - loss: 0.9948
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3450 - loss: 1.0059
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3446 - loss: 1.0021
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3487 - loss: 0.9913
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3452 - loss: 0.9935
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3445 - loss: 0.9986
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3550 - loss: 0.9856
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3492 - loss: 0.9782
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3534 - loss: 0.9880
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3558 - loss: 0.9874
Epoch 12/100
250/250 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
ypred = ann.predict(xtest)
ypred = ypred > 0.5
ypred

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[False],
       [ True],
       [False],
       ...,
       [False],
       [ True],
       [False]])

In [ ]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.82      0.42      0.56      1585
           1       0.23      0.66      0.34       415

    accuracy                           0.47      2000
   macro avg       0.53      0.54      0.45      2000
weighted avg       0.70      0.47      0.51      2000



In [ ]:
ann.evaluate(xtrain,ytrain)

250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4574 - loss: 0.7917


[0.7925887703895569, 0.4516249895095825]

# Hyperparameter tuning


In [ ]:
list(range(8,128,8))

[8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120]

In [ ]:
def hyper(hp):
    model = Sequential()

    # Add input layer
    model.add(Dense(hp.Int('units0', min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation0', values=['relu', 'tanh', 'sigmoid']),
                    input_dim=13))

    # Add additional hidden layers
    for i in range(1, hp.Int('num_layers', min_value=1, max_value=10)):
        model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                        activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])))

    # Add output layer
    model.add(Dense(units=1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
tuner = kf.RandomSearch(hyper,
                        objective = 'val_accuracy',
                        max_trials = 3)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner.search(xtrain,ytrain,epochs = 3, validation_data = (xtest,ytest))
# search() is similar to fit()

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.8195000290870667

Best val_accuracy So Far: 0.8460000157356262
Total elapsed time: 00h 00m 33s


In [ ]:
tuner.get_best_hyperparameters()[0].values # will give first model where accuracy is high
# Each of the 13 input features is connected to all 120 neurons in the input layer.
# This means that each of the 120 neurons will receive all 13 input values
# and apply a weighted sum and activation function to compute an output.

{'units0': 24,
 'activation0': 'tanh',
 'num_layers': 7,
 'optimizer': 'adam',
 'units1': 120,
 'activation1': 'relu',
 'units2': 128,
 'activation2': 'sigmoid',
 'units3': 96,
 'activation3': 'sigmoid',
 'units4': 88,
 'activation4': 'sigmoid',
 'units5': 8,
 'activation5': 'sigmoid',
 'units6': 56,
 'activation6': 'tanh',
 'units7': 72,
 'activation7': 'tanh',
 'units8': 88,
 'activation8': 'tanh'}

In [ ]:
tuned_model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
tuned_model.fit(xtrain,ytrain,epochs=50, validation_data=(xtest,ytest))

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8453 - loss: 0.3750 - val_accuracy: 0.8505 - val_loss: 0.3536
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8597 - loss: 0.3485 - val_accuracy: 0.8535 - val_loss: 0.3470
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8564 - loss: 0.3489 - val_accuracy: 0.8560 - val_loss: 0.3557
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8538 - loss: 0.3540 - val_accuracy: 0.8550 - val_loss: 0.3580
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8632 - loss: 0.3499 - val_accuracy: 0.8660 - val_loss: 0.3365
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8635 - loss: 0.3392 - val_accuracy: 0.8605 - val_loss: 0.3405
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8632 - loss: 0.3290 - val_accuracy: 0.8670 - val_loss: 0.3459
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8679 - loss: 0.3335 - val_accuracy: 

In [ ]:
tuned_model.evaluate(xtrain,ytrain) # Training accuracy

250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8865 - loss: 0.2742


[0.271078884601593, 0.887374997138977]

In [ ]:
tuned_model.evaluate(xtest,ytest)# Testing accuracy

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8551 - loss: 0.3756


[0.39238157868385315, 0.8514999747276306]